In [1]:
# !pip install requests-kerberos

In [2]:
# !pip install -r requirements.txt

In [14]:
import json
import getpass
import requests
from urllib.parse import urlparse

OSI_AF_ATTRIBUTE_TAG = 'Estado'
OSI_AF_DATABASE = 'PAE'
OSI_AF_ELEMENT = 'DLS-339'


def call_headers(include_content_type):
    """ Create API call headers
        @includeContentType boolean: Flag determines whether or not the
        content-type header is included
    """
    if include_content_type is True:
        header = {
            'content-type': 'application/json',
            'X-Requested-With': 'XmlHttpRequest'
        }
    else:
        header = {
            'X-Requested-With': 'XmlHttpRequest'
        }

    return header


def call_security_method(security_method, user_name, user_password):
    """ Create API call security method
        @param security_method string: Security method to use: basic or kerberos
        @param user_name string: The user's credentials name
        @param user_password string: The user's credentials password
    """
    from requests.auth import HTTPBasicAuth
    from requests_kerberos import HTTPKerberosAuth

    if security_method.lower() == 'basic':
        security_auth = HTTPBasicAuth(user_name, user_password)
    else:
        security_auth = HTTPKerberosAuth(mutual_authentication='REQUIRED',
                                         sanitize_mutual_error_response=False)

    return security_auth


def read_attribute_snapshot(piwebapi_url, asset_server, user_name, user_password,
                            piwebapi_security_method, verify_ssl):
    """ Read a single value
        @param piwebapi_url string: The URL of the PI Web API
        @param asset_server string: Name of the Asset Server
        @param user_name string: The user's credentials name
        @param user_password string: The user's credentials password
        @param piwebapi_security_method string: Security method: basic or kerberos
        @param verify_ssl: If certificate verification will be performed
    """
    print('readAttributeSnapshot')

    #  create security method - basic or kerberos
    security_method = call_security_method(
        piwebapi_security_method, user_name, user_password)

    #  Get the sample tag
    request_url = '{}/attributes?path=\\\\{}\\{}\\{}|{}'.format(
        piwebapi_url, asset_server, OSI_AF_DATABASE, OSI_AF_ELEMENT, OSI_AF_ATTRIBUTE_TAG)
    print(request_url)
    response = requests.get(request_url, auth=security_method, verify=verify_ssl)

    #  Only continue if the first request was successful
    if response.status_code == 200:
        print(response.text)
        #  Deserialize the JSON Response
        data = json.loads(response.text)

        url = urlparse(piwebapi_url + '/streams/' + data['WebId'] + '/value')
        # Validate URL
        assert url.scheme == 'https'
        assert url.geturl().startswith(piwebapi_url)

        #  Read the single stream value
        response = requests.get(url.geturl(),
                                auth=security_method, verify=verify_ssl)

        if response.status_code == 200:
            print('{} Snapshot Value'.format(OSI_AF_ATTRIBUTE_TAG))
            print(json.dumps(json.loads(response.text), indent=4, sort_keys=True))
        else:
            print(response.status_code, response.reason, response.text)
    else:
        print(response.status_code, response.reason, response.text)
    return response.status_code

In [30]:
%run appsettings.py
import os
os.environ['REQUESTS_CA_BUNDLE'] = r"C:\Users\scruple077\Documents\Certificates\pi-web-api-certificate.pem"

read_attribute_snapshot(PIWEBAPI_URL,AF_SERVER_NAME,USER_NAME,USER_PASSWORD,AUTH_TYPE,VERIFY_SSL)
print(AUTH_TYPE)

readAttributeSnapshot
401 Unauthorized {"Message":"Authorization has been denied for this request."}
basic


In [29]:
VERIFY_SSL

True

In [ ]:
import os
import openai

openai.api_key = input('')#os.getenv(input('Enter your OpenAI API key: '))

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="\"\"\"\n1. Create a list of first names\n2. Create a list of last names\n3. Combine them randomly into a list of 100 full names\n\"\"\"",
  temperature=1,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0)

In [ ]:
import base64

# Your username and password
username = 'mchicatun'
password = 'Justthewayyouare10'

# Encode the username and password
credentials = base64.b64encode(f'{username}:{password}'.encode('utf-8')).decode('utf-8')

# Headers for the API request
headers = {
    'Authorization': f'Basic {credentials}'
}


In [16]:
from langchain.chains import APIChain
from langchain import LLMChain#, TextRequestsWrapper
from langchain.chains.api.prompt import API_URL_PROMPT, API_RESPONSE_PROMPT
from langchain.llms import OpenAI

# Initialize OpenAI's DaVinci model
llm = OpenAI(openai_api_key=openai.api_key)#, engine='text-davinci-003')
os.environ['REQUESTS_CA_BUNDLE'] = r"C:\Users\scruple077\Documents\Certificates\pi-web-api-certificate.pem"

# API documentation as a string
# api_docs = "https://docs.aveva.com/bundle/pi-web-api/page/1023070.html"  # Replace with your actual API documentation
api_docs = "https://adetechindustrial.ddns.net/piwebapi"
# Headers for the API request
# headers = {
#     "Authorization": "Bearer your-api-key"
# }

# Initialize the APIChain
chain = APIChain.from_llm_and_api_docs(
    llm=llm,
    api_docs=api_docs,
    headers=headers,
    api_url_prompt=API_URL_PROMPT,
    api_response_prompt=API_RESPONSE_PROMPT
)

# Ask a question about the data
inputs = {
    "question": "Bring the first ten elements in PI Asset Framework"
}
response = chain._call(inputs)

# Print the response
print(response)

NameError: name 'openai' is not defined